In [33]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('egd').getOrCreate()

df_class = spark.read.format("csv").option("header", "false").option("inferSchema", "true").option("charset", "UTF-8").load("sentiment.csv")

cols = ['sentiment','id','date','query_string','user','text']
df_class = df_class.toDF(*cols)

In [30]:
import numpy as np
import matplotlib.pyplot as plt

df_class.show(5)

+---------+----------+--------------------+------------+---------------+--------------------+
|sentiment|        id|                date|query_string|           user|                text|
+---------+----------+--------------------+------------+---------------+--------------------+
|        0|1467810369|Mon Apr 06 22:19:...|    NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|        0|1467810672|Mon Apr 06 22:19:...|    NO_QUERY|  scotthamilton|is upset that he ...|
|        0|1467810917|Mon Apr 06 22:19:...|    NO_QUERY|       mattycus|@Kenichan I dived...|
|        0|1467811184|Mon Apr 06 22:19:...|    NO_QUERY|        ElleCTF|my whole body fee...|
|        0|1467811193|Mon Apr 06 22:19:...|    NO_QUERY|         Karoli|@nationwideclass ...|
+---------+----------+--------------------+------------+---------------+--------------------+
only showing top 5 rows



In [3]:
from pyspark.sql.functions import count

# assuming df is a Pyspark DataFrame
counts = df_class.groupBy("sentiment").agg(count("*").alias("count"))
counts.show()

+---------+------+
|sentiment| count|
+---------+------+
|        0|800000|
|        4|800000|
+---------+------+



In [4]:
df = df_class.drop("id", "date", "query_string", "user")

In [5]:
df.filter(df.sentiment == 0).take(10)

[Row(sentiment=0, text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"),
 Row(sentiment=0, text="is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"),
 Row(sentiment=0, text='@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'),
 Row(sentiment=0, text='my whole body feels itchy and like its on fire '),
 Row(sentiment=0, text="@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "),
 Row(sentiment=0, text='@Kwesidei not the whole crew '),
 Row(sentiment=0, text='Need a hug '),
 Row(sentiment=0, text="@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?"),
 Row(sentiment=0, text="@Tatiana_K nope they didn't have it "),
 Row(sentiment=0, text='@twittera que me muera ? ')]

In [6]:
df.filter(df.sentiment == 4).take(10)

[Row(sentiment=4, text='I LOVE @Health4UandPets u guys r the best!! '),
 Row(sentiment=4, text='im meeting up with one of my besties tonight! Cant wait!!  - GIRL TALK!!'),
 Row(sentiment=4, text='@DaRealSunisaKim Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart. '),
 Row(sentiment=4, text='Being sick can be really cheap when it hurts too much to eat real food  Plus, your friends make you soup'),
 Row(sentiment=4, text='@LovesBrooklyn2 he has that effect on everyone '),
 Row(sentiment=4, text='@ProductOfFear You can tell him that I just burst out laughing really loud because of that  Thanks for making me come out of my sulk!'),
 Row(sentiment=4, text='@r_keith_hill Thans for your response. Ihad already find this answer '),
 Row(sentiment=4, text="@KeepinUpWKris I am so jealous, hope you had a great time in vegas! how did you like the ACM's?! LOVE YOUR SHOW!! "),
 Row(sentiment=4, text='@tommcfly ah, congrats mr fletch

<b>Data Preparation</b>

In [10]:
from pyspark.sql.functions import length

df = df.withColumn("pre_clean_len", length(df.text))
df.show(10)

+---------+--------------------+-------------+
|sentiment|                text|pre_clean_len|
+---------+--------------------+-------------+
|        0|@switchfoot http:...|          115|
|        0|is upset that he ...|          111|
|        0|@Kenichan I dived...|           89|
|        0|my whole body fee...|           47|
|        0|@nationwideclass ...|          111|
|        0|@Kwesidei not the...|           29|
|        0|         Need a hug |           11|
|        0|@LOLTrish hey  lo...|           99|
|        0|@Tatiana_K nope t...|           36|
|        0|@twittera que me ...|           25|
+---------+--------------------+-------------+
only showing top 10 rows



In [19]:
import matplotlib.pyplot as plt

pre_clean_len_list = df.select("pre_clean_len").rdd.flatMap(lambda x: x).collect()
plt.boxplot(pre_clean_len_list)
plt.show()

23/05/01 16:07:05 ERROR Executor: Exception in task 0.0 in stage 15.0 (TID 45)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 683, in main
    raise RuntimeError(
RuntimeError: Python in worker has different version 3.10 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:561)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:767)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:749)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:514)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 45) (172.16.21.64 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 683, in main
    raise RuntimeError(
RuntimeError: Python in worker has different version 3.10 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:561)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:767)
	at org.apache.spark.api.python.PythonRu

In [20]:
df.filter(df.pre_clean_len > 140).show(10)

+---------+--------------------+-------------+
|sentiment|                text|pre_clean_len|
+---------+--------------------+-------------+
|        0|Awwh babs... you ...|          142|
|        0|Whinging. My clie...|          145|
|        0|@TheLeagueSF Not ...|          145|
|        0|#3 woke up and wa...|          144|
|        0|My bathtub drain ...|          146|
|        0|pears &amp; Brie,...|          150|
|        0|Have an invite fo...|          141|
|        0|Damnit I was real...|          141|
|        0|Why do I keep loo...|          141|
|        0|Used the term &qu...|          148|
+---------+--------------------+-------------+
only showing top 10 rows



<b>Data Preparation 1: HTML decoding</b>

In [21]:
text_row = df.select("text").collect()[279][0]
print(text_row)

Whinging. My client&amp;boss don't understand English well. Rewrote some text unreadable. It's written by v. good writer&amp;reviewed correctly. 


In [22]:
#pip install lxml

In [15]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from bs4 import BeautifulSoup

def parse_html(text):
    soup = BeautifulSoup(text, 'lxml')
    return soup.get_text()

parse_html_udf = udf(parse_html, StringType())
df.withColumn("text_parsed", parse_html_udf("text")).select("text_parsed").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------+
|text_parsed                                                                                                          |
+---------------------------------------------------------------------------------------------------------------------+
|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  |
|is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!      |
|@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds                            |
|my whole body feels itchy and like its on fire                                                                       |
|@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.       |
|@Kwesidei not the whole crew           

/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


<b>Data Preparation 2: ‘@’mention</b>

In [25]:
text_row = df.select("text").collect()[343][0]
print(text_row)

@TheLeagueSF Not Fun &amp; Furious? The new mantra for the Bay 2 Breakers? It was getting 2 rambunctious;the city overreacted &amp; clamped down 


In [32]:
import os

os.environ['PYSPARK_PYTHON'] = '/path/to/python3.10'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/path/to/python3.10'


In [34]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re

def remove_mentions(text):
    return re.sub(r'@[A-Za-z0-9]+', '', text)

remove_mentions_udf = udf(remove_mentions, StringType())
df.withColumn("text_clean", remove_mentions_udf("text")).select("text_clean").show(truncate=False)

23/05/01 16:13:22 ERROR Executor: Exception in task 0.0 in stage 29.0 (TID 101)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 683, in main
    raise RuntimeError(
RuntimeError: Python in worker has different version 3.10 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:561)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:514)
	at org.apache.spark.InterruptibleIterator.hasNext(I

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 683, in main
    raise RuntimeError(
RuntimeError: Python in worker has different version 3.10 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.


<b>Data Preparation 3: URL links</b>

In [18]:
text_row = df.select("text").collect()[0][0]
print(text_row)

@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D


In [16]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re

def remove_urls(text):
    return re.sub('https?://[A-Za-z0-9./]+', '', text)

remove_urls_udf = udf(remove_urls, StringType())
df.withColumn("text_clean", remove_urls_udf("text")).select("text_clean").show(truncate=False)

23/05/01 16:05:53 ERROR Executor: Exception in task 0.0 in stage 14.0 (TID 44)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 683, in main
    raise RuntimeError(
RuntimeError: Python in worker has different version 3.10 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:561)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:514)
	at org.apache.spark.InterruptibleIterator.hasNext(In

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 683, in main
    raise RuntimeError(
RuntimeError: Python in worker has different version 3.10 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.


<b>Data Preparation 4: UTF-8 BOM (Byte Order Mark)</b>

In [26]:
row = df.select('text').collect()[226]
text_value = row['text']

In [21]:
testing = text_value.encode("utf-8-sig")
testing

b'\xef\xbb\xbfTuesday\xef\xbf\xbdll start with reflection \xef\xbf\xbdn then a lecture in Stress reducing techniques. That sure might become very useful for us accompaniers '

In [22]:
testing = testing.decode("utf-8-sig")
testing

'Tuesday�ll start with reflection �n then a lecture in Stress reducing techniques. That sure might become very useful for us accompaniers '

In [23]:
#testing.replace(u"\ufffd", "?")

In [27]:
# Remove non-ASCII characters
testing = re.sub(r'[^\x00-\x7F]+', '?', testing)
testing

'Tuesday?ll start with reflection ?n then a lecture in Stress reducing techniques. That sure might become very useful for us accompaniers '

<b>Data Preparation 5: hashtag / numbers</b>

In [28]:
txt = df.select('text').collect()[175]
txt

Row(text="@machineplay I'm so sorry you're having to go through this. Again.  #therapyfail")

In [30]:
re.sub("[^a-zA-Z]", " ", txt['text'])

' machineplay I m so sorry you re having to go through this  Again    therapyfail'

In [31]:
#pip install nltk

<b>Now in one function with enhancment from Part II</b>

In [32]:
from bs4 import BeautifulSoup
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import nltk
nltk.download('punkt')
tok = nltk.RegexpTokenizer(r'\w+')
pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't": "is not", "aren't": "are not", "wasn't": "was not", "weren't": "were not",
                 "haven't": "have not","hasn't": "has not","hadn't": "had not","won't": "will not",
                 "wouldn't": "would not","don't": "do not","doesn't": "does not","didn't": "did not",
                 "can't": "can not","couldn't": "could not","shouldn't": "should not","mightn't": "might not",
                 "mustn't": "must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.encode("utf-8-sig")
        bom_removed = bom_removed.decode("utf-8-sig").replace("[^\x00-\x7F]+","?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

tweet_cleaner_udf = udf(tweet_cleaner, StringType())

[nltk_data] Downloading package punkt to /Users/zakirasa/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [33]:
from pyspark.sql.functions import monotonically_increasing_id
df_with_index = df.rdd.zipWithIndex().toDF(["data", "index"]).select("data.*", "index")

In [34]:
df_with_index.show()

+---------+--------------------+-------------+-----+
|sentiment|                text|pre_clean_len|index|
+---------+--------------------+-------------+-----+
|        0|@switchfoot http:...|          115|    0|
|        0|is upset that he ...|          111|    1|
|        0|@Kenichan I dived...|           89|    2|
|        0|my whole body fee...|           47|    3|
|        0|@nationwideclass ...|          111|    4|
|        0|@Kwesidei not the...|           29|    5|
|        0|         Need a hug |           11|    6|
|        0|@LOLTrish hey  lo...|           99|    7|
|        0|@Tatiana_K nope t...|           36|    8|
|        0|@twittera que me ...|           25|    9|
|        0|spring break in p...|           43|   10|
|        0|I just re-pierced...|           26|   11|
|        0|@caregiving I cou...|           94|   12|
|        0|@octolinz16 It it...|           77|   13|
|        0|@smarrison i woul...|          117|   14|
|        0|@iamjazzyfizzle I...|          103|

In [35]:
clean_tweet_udf = udf(tweet_cleaner, StringType())

df22 = df.withColumn("cleaned_text", clean_tweet_udf("text"))
df22.show()

+---------+--------------------+-------------+--------------------+
|sentiment|                text|pre_clean_len|        cleaned_text|
+---------+--------------------+-------------+--------------------+
|        0|@switchfoot http:...|          115|awww that bummer ...|
|        0|is upset that he ...|          111|is upset that he ...|
|        0|@Kenichan I dived...|           89|dived many times ...|
|        0|my whole body fee...|           47|my whole body fee...|
|        0|@nationwideclass ...|          111|no it not behavin...|
|        0|@Kwesidei not the...|           29|  not the whole crew|
|        0|         Need a hug |           11|            need hug|
|        0|@LOLTrish hey  lo...|           99|hey long time no ...|
|        0|@Tatiana_K nope t...|           36|nope they did not...|
|        0|@twittera que me ...|           25|        que me muera|
|        0|spring break in p...|           43|spring break in p...|
|        0|I just re-pierced...|           26|ju

/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [36]:
df22.show(5)

+---------+--------------------+-------------+--------------------+
|sentiment|                text|pre_clean_len|        cleaned_text|
+---------+--------------------+-------------+--------------------+
|        0|@switchfoot http:...|          115|awww that bummer ...|
|        0|is upset that he ...|          111|is upset that he ...|
|        0|@Kenichan I dived...|           89|dived many times ...|
|        0|my whole body fee...|           47|my whole body fee...|
|        0|@nationwideclass ...|          111|no it not behavin...|
+---------+--------------------+-------------+--------------------+
only showing top 5 rows



/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [18]:
# Note that df is not changed as Dataframes are immutable
Other_df=df22.na.drop()
Other_df.show()

NameError: name 'df22' is not defined

In [38]:
df22.filter(df.text.isNull()).show()

+---------+----+-------------+------------+
|sentiment|text|pre_clean_len|cleaned_text|
+---------+----+-------------+------------+
+---------+----+-------------+------------+



In [39]:
df22.filter(df.sentiment.isNull()).show()

+---------+----+-------------+------------+
|sentiment|text|pre_clean_len|cleaned_text|
+---------+----+-------------+------------+
+---------+----+-------------+------------+



In [46]:
df22 = df22.drop("text","pre_clean_len","processed_text")

In [47]:
df22.show(5)

+---------+--------------------+
|sentiment|        cleaned_text|
+---------+--------------------+
|        0|awww that bummer ...|
|        0|is upset that he ...|
|        0|dived many times ...|
|        0|my whole body fee...|
|        0|no it not behavin...|
+---------+--------------------+
only showing top 5 rows



/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [43]:
# Save the DataFrame to a CSV file
#df22.write.option("header", "true").csv("cleaned_dataWithZaki.csv")

In [48]:
df_filtered = df22.filter(df22.sentiment == 0)
df_filtered.show(truncate=False)

+---------+---------------------------------------------------------------------------------------------------------+
|sentiment|cleaned_text                                                                                             |
+---------+---------------------------------------------------------------------------------------------------------+
|0        |awww that bummer you shoulda got david carr of third day to do it                                        |
|0        |is upset that he can not update his facebook by texting it and might cry as result school today also blah|
|0        |dived many times for the ball managed to save the rest go out of bounds                                  |
|0        |my whole body feels itchy and like its on fire                                                           |
|0        |no it not behaving at all mad why am here because can not see you all over there                         |
|0        |not the whole crew                           

/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


<b>Machine Learning</b>

In [49]:
sample_df = df22.sample(fraction=0.1, seed=42)

In [50]:
    """data = df22.select('cleaned_text', 'sentiment').filter("cleaned_text is not NULL and sentiment is not NULL")

    # Feature extraction
    tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="words")
    wordsData = tokenizer.transform(data)

    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=1000)
    featurizedData = hashingTF.transform(wordsData)

    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfModel = idf.fit(featurizedData)
    rescaledData = idfModel.transform(featurizedData)

    # Split the data
    (trainingData, testData) = rescaledData.randomSplit([0.7, 0.3], seed=100)

    # Train the model
    nb = NaiveBayes(labelCol="sentiment", featuresCol="features", smoothing=1.0, modelType="multinomial")
    model = nb.fit(trainingData)

    # Evaluate the model
    evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="sentiment")
    predictions = model.transform(testData)"""

'data = df22.select(\'cleaned_text\', \'sentiment\').filter("cleaned_text is not NULL and sentiment is not NULL")\n\n# Feature extraction\ntokenizer = Tokenizer(inputCol="cleaned_text", outputCol="words")\nwordsData = tokenizer.transform(data)\n\nhashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=1000)\nfeaturizedData = hashingTF.transform(wordsData)\n\nidf = IDF(inputCol="rawFeatures", outputCol="features")\nidfModel = idf.fit(featurizedData)\nrescaledData = idfModel.transform(featurizedData)\n\n# Split the data\n(trainingData, testData) = rescaledData.randomSplit([0.7, 0.3], seed=100)\n\n# Train the model\nnb = NaiveBayes(labelCol="sentiment", featuresCol="features", smoothing=1.0, modelType="multinomial")\nmodel = nb.fit(trainingData)\n\n# Evaluate the model\nevaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="sentiment")\npredictions = model.transform(testData)'

In [51]:
#predictions

In [52]:
#predictions.withColumn('sentiment', col('sentiment').cast('float'))

In [ ]:
#kd = df22.select('cleaned_text', predictions.withColumn('sentiment', col('sentiment').cast('float'))).filter("cleaned_text is not NULL and sentiment is not NULL")

In [60]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import col
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import HashingTF
from pyspark.ml.feature import IDF
from pyspark.ml.classification import NaiveBayes




data = df22.select('cleaned_text', 'sentiment').filter("cleaned_text is not NULL and sentiment is not NULL")

# Feature extraction
tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="words")
wordsData = tokenizer.transform(data)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=1000)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

# Split the data
(trainingData, testData) = rescaledData.randomSplit([0.7, 0.3], seed=100)

# Train the model
nb = NaiveBayes(labelCol="sentiment", featuresCol="features", smoothing=1.0, modelType="multinomial")
model = nb.fit(trainingData)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="sentiment")
predictions = model.transform(testData)
#ADDED BY KARIM
predictions = predictions.withColumn('sentiment', col('sentiment').cast('float'))

# Compute evaluation metrics
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

# Compute precision, recall, f1-score and AUC
labels = predictions.select('sentiment').distinct().rdd.flatMap(lambda x: x).collect()
metrics = MulticlassMetrics(predictions.select("prediction", "sentiment").rdd)

for label in labels:
    print(f"Class {label}:")
    print("Precision:", metrics.precision(label))
    print("Recall:", metrics.recall(label))
    print("F1-score:", metrics.fMeasure(label))
      
# Compute AUC
binary_evaluator = evaluator.setMetricName("accuracy").setLabelCol("sentiment")
auc = binary_evaluator.evaluate(predictions)
print("AUC:", auc)

# Compute confusion matrix
print("Confusion Matrix:")
confusion_matrix = metrics.confusionMatrix().toArray()
for i in range(len(labels)):
    for j in range(len(labels)):
        print(confusion_matrix[i][j], end='\t')
    print()

/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  war

Accuracy: 0.35654723985700193


/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Class 0.0:


Precision: 0.7094775087219504
Recall: 0.7160823599518276
F1-score: 0.7127646336607381
Class 4.0:
Precision: 0.0
Recall: 0.0
F1-score: 0.0


AUC: 0.35820687067456014
Confusion Matrix:
171839.0	0.0	
70366.0	0.0	


In [66]:
# Compute AUC
#binary_evaluator = evaluator.setMetricName("areaUnderROC").setLabelCol("label").setMetricLabel(1.0)
#auc = binary_evaluator.evaluate(predictions)
#print("AUC:", auc)

# Compute AUC

binary_evaluator = evaluator.setMetricName("accuracy").setLabelCol("sentiment")
auc = binary_evaluator.evaluate(predictions)
print("AUC:", auc)


# Compute confusion matrix
print("Confusion Matrix:")
confusion_matrix = metrics.confusionMatrix().toArray()
for i in range(len(labels)):
    for j in range(len(labels)):
        print(confusion_matrix[i][j], end='\t')
    print()

/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  war

AUC: 0.35820687067456014
Confusion Matrix:
171839.0	0.0	
70366.0	0.0	


<i>This code performs k-fold cross-validation with 5 folds, and uses a grid search over different values of the smoothing parameter for the Naive Bayes model. The CrossValidator class trains and evaluates the model on each fold, and returns the best model found. Finally, the best model is evaluated on the test data using the MulticlassClassificationEvaluator.</i>

<b>Part Three III</b>

In [13]:
#pip install findspark

In [105]:
# cmd nproc

In [17]:
try:
    # create SparkContext on all CPUs available: in my case I have 4 CPUs on my laptop
    sc = ps.SparkContext('local[4]')
    sqlContext = SQLContext(sc)
    print("Just created a SparkContext")
except ValueError:
    warnings.warn("SparkContext already exists in this scope")

NameError: name 'ps' is not defined

In [15]:
df22.show(5)

NameError: name 'df22' is not defined

In [128]:
import findspark
findspark.init()
import pyspark as ps
import warnings
from pyspark.sql import SQLContext

In [134]:
try:
    # create SparkContext on all CPUs available: in my case I have 4 CPUs on my laptop
    sc = ps.SparkContext('local[4]')
    sqlContext = SQLContext(sc)
    print("Just created a SparkContext")
except ValueError:
    warnings.warn("SparkContext already exists in this scope")

/tmp/ipykernel_15536/1224509973.py:7: UserWarning: SparkContext already exists in this scope
  warnings.warn("SparkContext already exists in this scope")


In [135]:
import pyspark as ps
from pyspark.sql import SQLContext
import warnings

try:
    # Check if SparkContext already exists
    sc = ps.SparkContext.getOrCreate()
    sqlContext = SQLContext(sc)
    print("Using existing SparkContext")
except ValueError:
    warnings.warn("SparkContext already exists in this scope")

Using existing SparkContext


/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [141]:
df22.printSchema()

root
 |-- sentiment: integer (nullable = true)
 |-- cleaned_text: string (nullable = true)



In [139]:
df_k = df22

In [142]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField('sentiment', IntegerType(), True),
    StructField('cleaned_text', StringType(), True)
])

df_k = sqlContext.createDataFrame(df22, schema)

TypeError: data is already a DataFrame

In [143]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

# Assuming df_k is already created using df22 and a schema
#df_k = ...

# Use df_k with SQLContext
df_k.registerTempTable("my_table")
results = sqlContext.sql("SELECT * FROM my_table")

/usr/local/spark/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [145]:
results.show(5)

+---------+--------------------+
|sentiment|        cleaned_text|
+---------+--------------------+
|        0|awww that bummer ...|
|        0|is upset that he ...|
|        0|dived many times ...|
|        0|my whole body fee...|
|        0|no it not behavin...|
+---------+--------------------+
only showing top 5 rows



In [ ]:
df_k = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(df22)
type(df_k)

In [ ]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(df22)
type(df)

In [110]:
df_III = df22
type(df_III)

pyspark.sql.dataframe.DataFrame

In [112]:
df_III.show(5)

+---------+--------------------+
|sentiment|        cleaned_text|
+---------+--------------------+
|        0|awww that bummer ...|
|        0|is upset that he ...|
|        0|dived many times ...|
|        0|my whole body fee...|
|        0|no it not behavin...|
+---------+--------------------+
only showing top 5 rows



In [113]:
df_III = df_III.dropna()
df_III.count()

1600000

In [124]:
from pyspark.sql.functions import monotonically_increasing_id

# Create a sample DataFrame


# Add an index column to the DataFrame
df_with_index = df_III.withColumn('index', monotonically_increasing_id())

# Show the DataFrame with the index column
df_with_index.show()

+---------+--------------------+-----+
|sentiment|        cleaned_text|index|
+---------+--------------------+-----+
|        0|awww that bummer ...|    0|
|        0|is upset that he ...|    1|
|        0|dived many times ...|    2|
|        0|my whole body fee...|    3|
|        0|no it not behavin...|    4|
|        0|  not the whole crew|    5|
|        0|            need hug|    6|
|        0|hey long time no ...|    7|
|        0|nope they did not...|    8|
|        0|        que me muera|    9|
|        0|spring break in p...|   10|
|        0|just re pierced m...|   11|
|        0|could not bear to...|   12|
|        0|it it counts idk ...|   13|
|        0|would ve been the...|   14|
|        0|wish got to watch...|   15|
|        0|hollis death scen...|   16|
|        0| about to file taxes|   17|
|        0|ahh ive always wa...|   18|
|        0|oh dear were you ...|   19|
+---------+--------------------+-----+
only showing top 20 rows



In [146]:
(train_set, val_set, test_set) = results.randomSplit([0.98, 0.01, 0.01], seed = 2000)

<b>HashingTF + IDF + Logistic Regression</b>

In [147]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="words")
hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx = StringIndexer(inputCol = "sentiment", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer, hashtf, idf, label_stringIdx])

pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
val_df = pipelineFit.transform(val_set)
train_df.show(5)

+---------+------------+-----+--------------------+--------------------+-----+
|sentiment|cleaned_text|words|                  tf|            features|label|
+---------+------------+-----+--------------------+--------------------+-----+
|        0|            |   []|(65536,[52572],[1...|(65536,[52572],[6...|  0.0|
|        0|            |   []|(65536,[52572],[1...|(65536,[52572],[6...|  0.0|
|        0|            |   []|(65536,[52572],[1...|(65536,[52572],[6...|  0.0|
|        0|            |   []|(65536,[52572],[1...|(65536,[52572],[6...|  0.0|
|        0|            |   []|(65536,[52572],[1...|(65536,[52572],[6...|  0.0|
+---------+------------+-----+--------------------+--------------------+-----+
only showing top 5 rows



In [148]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=100)
lrModel = lr.fit(train_df)
predictions = lrModel.transform(val_df)

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.8585199971013485

In [149]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(val_set.count())
accuracy

0.7865394274234053

In [67]:
#using the Light GBM:

!pip install lightgbm


     |████████████████████████████████| 1.3 MB 2.0 MB/s eta 0:00:01


In [71]:
import lightgbm as lgb
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import VectorUDT


OSError: dlopen(/Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <D21A7969-4567-3BC7-94ED-6A9E83AE9D78> /Users/zakirasa/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

In [ ]:
import lightgbm as lgb
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import VectorUDT


In [ ]:
#want to stop the process, buecasue it took long time- so far. 

import sys

# Code execution...

# Stop the current running process
sys.exit()


In [ ]:
import os
os.environ['DYLD_FALLBACK_LIBRARY_PATH'] = '/opt/anaconda3/lib'


<b>CountVectorizer + IDF + Logistic Regression</b>

In [ ]:
%%time
from pyspark.ml.feature import CountVectorizer

tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="words")
cv = CountVectorizer(vocabSize=2**16, inputCol="words", outputCol='cv')
idf = IDF(inputCol='cv', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
label_stringIdx = StringIndexer(inputCol = "sentiment", outputCol = "label")
lr = LogisticRegression(maxIter=100)
pipeline = Pipeline(stages=[tokenizer, cv, idf, label_stringIdx, lr])

pipelineFit = pipeline.fit(train_set)
predictions = pipelineFit.transform(val_set)
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(val_set.count())
roc_auc = evaluator.evaluate(predictions)

print ("Accuracy Score: {0:.4f}".format(accuracy))
print ("ROC-AUC: {0:.4f}".format(roc_auc))

<b>N-gram Implementation<br>Do not run the two cells underneath they will take forever</b>

In [162]:
"""from pyspark.ml.feature import NGram, VectorAssembler
from pyspark.ml.feature import ChiSqSelector

def build_trigrams(inputCol=["cleaned_text","sentiment"], n=3):
    tokenizer = [Tokenizer(inputCol="cleaned_text", outputCol="words")]
    ngrams = [
        NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))
        for i in range(1, n + 1)
    ]

    cv = [
        CountVectorizer(vocabSize=2**14,inputCol="{0}_grams".format(i),
            outputCol="{0}_tf".format(i))
        for i in range(1, n + 1)
    ]
    idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]

    assembler = [VectorAssembler(
        inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],
        outputCol="rawFeatures"
    )]
    label_stringIdx = [StringIndexer(inputCol = "sentiment", outputCol = "label")]
    selector = [ChiSqSelector(numTopFeatures=2**14,featuresCol='rawFeatures', outputCol="features")]
    lr = [LogisticRegression(maxIter=100)]
    return Pipeline(stages=tokenizer + ngrams + cv + idf+ assembler + label_stringIdx+selector+lr)"""

'from pyspark.ml.feature import NGram, VectorAssembler\nfrom pyspark.ml.feature import ChiSqSelector\n\ndef build_trigrams(inputCol=["cleaned_text","sentiment"], n=3):\n    tokenizer = [Tokenizer(inputCol="cleaned_text", outputCol="words")]\n    ngrams = [\n        NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))\n        for i in range(1, n + 1)\n    ]\n\n    cv = [\n        CountVectorizer(vocabSize=2**14,inputCol="{0}_grams".format(i),\n            outputCol="{0}_tf".format(i))\n        for i in range(1, n + 1)\n    ]\n    idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]\n\n    assembler = [VectorAssembler(\n        inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],\n        outputCol="rawFeatures"\n    )]\n    label_stringIdx = [StringIndexer(inputCol = "sentiment", outputCol = "label")]\n    selector = [ChiSqSelector(numTopFeatures=2**14,featuresCol=\'rawFeatures\', outputCol="features")]\n    lr

In [163]:
"""%%time
trigram_pipelineFit = build_trigrams().fit(train_set)
predictions = trigram_pipelineFit.transform(val_set)
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(dev_set.count())
roc_auc = evaluator.evaluate(predictions)# print accuracy, roc_auc
print("Accuracy Score: {0:.4f}".format(accuracy))
print("ROC-AUC: {0:.4f}".format(roc_auc))"""

'%%time\ntrigram_pipelineFit = build_trigrams().fit(train_set)\npredictions = trigram_pipelineFit.transform(val_set)\naccuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(dev_set.count())\nroc_auc = evaluator.evaluate(predictions)# print accuracy, roc_auc\nprint("Accuracy Score: {0:.4f}".format(accuracy))\nprint("ROC-AUC: {0:.4f}".format(roc_auc))'

In [159]:
from pyspark.ml.feature import NGram, VectorAssembler

def build_ngrams_wocs(inputCol=["cleaned_text","sentiment"], n=3):
    tokenizer = [Tokenizer(inputCol="cleaned_text", outputCol="words")]
    ngrams = [
        NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))
        for i in range(1, n + 1)
    ]

    cv = [
        CountVectorizer(vocabSize=5460,inputCol="{0}_grams".format(i),
            outputCol="{0}_tf".format(i))
        for i in range(1, n + 1)
    ]
    idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]

    assembler = [VectorAssembler(
        inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],
        outputCol="features"
    )]
    label_stringIdx = [StringIndexer(inputCol = "sentiment", outputCol = "label")]
    lr = [LogisticRegression(maxIter=100)]
    return Pipeline(stages=tokenizer + ngrams + cv + idf+ assembler + label_stringIdx+lr)

In [160]:
%%time

trigramwocs_pipelineFit = build_ngrams_wocs().fit(train_set)
predictions_wocs = trigramwocs_pipelineFit.transform(val_set)
accuracy_wocs = predictions_wocs.filter(predictions_wocs.label == predictions_wocs.prediction).count() / float(val_set.count())
roc_auc_wocs = evaluator.evaluate(predictions_wocs)# print accuracy, roc_auc

print("Accuracy Score: {0:.4f}".format(accuracy_wocs))
print("ROC-AUC: {0:.4f}".format(roc_auc_wocs))

Accuracy Score: 0.8104
ROC-AUC: 0.8825
CPU times: user 482 ms, sys: 288 ms, total: 771 ms
Wall time: 14min 16s


In [161]:
test_predictions = trigramwocs_pipelineFit.transform(test_set)
test_accuracy = test_predictions.filter(test_predictions.label == test_predictions.prediction).count() / float(test_set.count())
test_roc_auc = evaluator.evaluate(test_predictions)# print accuracy, roc_auc
print("Accuracy Score: {0:.4f}".format(test_accuracy))
print("ROC-AUC: {0:.4f}".format(test_roc_auc))

Accuracy Score: 0.8092
ROC-AUC: 0.8825


In [38]:
#sample_df = df22.sample(fraction=0.1, seed=42)

In [40]:
"""from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col

data = sample_df.select('cleaned_text', 'sentiment').filter("cleaned_text is not NULL and sentiment is not NULL")

# Feature extraction
tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="words")
wordsData = tokenizer.transform(data)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=1000)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

# Split the data
(trainingData, testData) = rescaledData.randomSplit([0.7, 0.3], seed=100)

# Train the model using k-fold cross-validation
nb = NaiveBayes(labelCol="sentiment", featuresCol="features", smoothing=1.0, modelType="multinomial")
paramGrid = ParamGridBuilder().addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]).build()
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="sentiment")
crossval = CrossValidator(estimator=nb, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cvModel = crossval.fit(trainingData)

# Evaluate the model on test data
predictions = cvModel.transform(testData)
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)"""

Accuracy: 0.35524547988536737


<b>Saving cleaned data as csv</b>

<b>PART TWO</b>

In [ ]:
# print the schema of the DataFrame
df22.printSchema()

# describe the DataFrame
df22.describe().show()

In [ ]:
from pyspark.sql.functions import isnull
# filter rows with null values
null_df = df22.filter(isnull("cleaned_text"))

# show the first few rows of the filtered DataFrame
null_df.show()

In [ ]:
from pyspark.sql.functions import isnull, col, sum as pyspark_sum

null_count = df22.select([pyspark_sum(col(c).isNull().cast("int")).alias(c) for c in df22.columns]).first()

# print the count
print(null_count)

In [ ]:
from pyspark.sql.functions import isnull, col

# check which columns have null values
null_cols = [c for c in df22.columns if df22.select(isnull(col(c))).collect()[0][0]]

# print the columns with null values
print(null_cols)

In [ ]:
# check which columns have null values
null_cols = [c for c in df22.columns if df22.select(isnull(col(c))).first()[0]]

# print the columns with null values
print(null_cols)

In [ ]:
from pyspark.sql.functions import col, monotonically_increasing_id

# add an index column to the DataFrame
df33 = df22.withColumn("index", monotonically_increasing_id())

# drop rows with null or NaN values
df33 = df33.na.drop()

# reset the index of the DataFrame
df33 = df33.withColumn("index", col("index").cast("int"))
df33 = df33.withColumn("index", col("index"))

# show the schema and first few rows of the resulting DataFrame
df33.printSchema()
df33.show(5)

In [ ]:
#!pip install wordcloud

from wordcloud import WordCloud

In [ ]:
import pandas as pd

# convert PySpark DataFrame to Pandas DataFrame
my_df_pd = df33.toPandas()

# filter negative tweets
neg_tweets = my_df_pd[my_df_pd.sentiment == 0]

# concatenate negative tweet strings
neg_string = ' '.join(neg_tweets['cleaned_text'].tolist())

# generate word cloud
wordcloud = WordCloud(width=1600, height=800, max_font_size=200).generate(neg_string)

# display word cloud
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# convert PySpark DataFrame to Pandas DataFrame
my_df_pd = df33.toPandas()

# filter positive tweets
pos_tweets = my_df_pd[my_df_pd.sentiment == 4]

# concatenate positive tweet strings
pos_string = ' '.join(pos_tweets['cleaned_text'].tolist())

# generate word cloud
wordcloud = WordCloud(width=1600, height=800, max_font_size=200, colormap='magma').generate(pos_string)

# display word cloud
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# convert PySpark DataFrame to Pandas DataFrame
my_df_pd = df33.toPandas()

# convert text column to list of strings
text_list = my_df_pd['cleaned_text'].tolist()

# initialize CountVectorizer
cvec = CountVectorizer()

# fit CountVectorizer to text data
cvec.fit(text_list)

In [ ]:
len(cvec.get_feature_names_out())

In [ ]:
#pip install --upgrade scikit-learn

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
print(df.columns)

In [ ]:
print(df33.head())

In [ ]:
# Transform the result into a pandas dataframe
df_class_pd = df33.toPandas()

neg_doc_matrix = cvec.transform(df_class_pd[df_class_pd.sentiment == 0].cleaned_text)
pos_doc_matrix = cvec.transform(df_class_pd[df_class_pd.sentiment == 4].cleaned_text)
neg_tf = np.sum(neg_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))
term_freq_df = pd.DataFrame([neg,pos],columns=cvec.get_feature_names_out()).transpose()

In [ ]:
term_freq_df.columns = ['negative', 'positive']
term_freq_df['total'] = term_freq_df['negative'] + term_freq_df['positive']
term_freq_df.sort_values(by='total', ascending=False).iloc[:10]